# Текущий подход по данным с дискретностью 1 мин
(Минутные данные взяты для тестирования и проработки подхода) 


По модели Avellaneda-Stoikov делается расчет 
- реализованная цена
$$r = S_t -q\gamma\sigma^2(T-t)$$
- спред
$$\delta_a + \delta_b = \gamma\sigma^2(T-t) + \frac{1}{\gamma}\ln(1+\frac{\gamma}{k})$$

### Параметр оптимизации (ищем RL методами), реализованный в текущем подходе
- $\gamma$ - уровень аверсии (ниприятия) к риску

##### Какие еще параметры можем оптимизировать
- T - время горизонта
- k - параметр интенсивности сделок, который находится из соотношения $\lambda(\delta) = A\exp(-k\delta)$ (это если не находим этот параметр из реальных данных)
- $\Delta k$ поправку в параметр. Здесь мы принимаем, что $k = \hat{k} + \Delta k$, где $\hat{k}$ оценка параметра из данных
- $\Delta\sigma$ поправку в волатильность. Здесь мы принимаем, что $\sigma = \hat\sigma + \Delta\sigma$, где $\hat\sigma$ оценка волатильности из данных

## Дальнейшие улучшения 
1. Реализация на более высокочастотных таймфреймах
2. Использовать книгу заказов в качестве observation для агента (скорее всего это сильно улучшит предсказательную способность модели)
3. Реализовать RL схему, которая учит на прямую определять реализованную цену и спред (скорее всего результаты будут не устойчивые, но проверить можно, если давать агенту книгу заказов)
4. Использовать более реалистичные модели рынка, например модель Мертона со скачками
$$ dS_t = \mu dt +\sigma dW +JdN_t$$
в которой резервированная цена считается по формуле
$$r = S_t -q\gamma\sigma^2 (T-t) - \lambda \mathbb{E}[e^{-\gamma J} - 1](T-t)$$

In [9]:
import math
import random
from collections import deque, namedtuple

import gym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from binance import Client
from gym import spaces


Get data

In [ ]:
# Инициализация клиента (без API ключей для публичных данных)
client = Client()


# Загрузка исторических данных
def get_klines(
    symbol="ETHUSDT",
    interval="1m",
    start_date="6 day ago UTC",
    end_date="2 day ago UTC",
):
    try:
        # Получение данных
        klines = client.get_historical_klines(
            symbol=symbol, interval=interval, start_str=start_date, end_str=end_date
        )

        # Преобразование в DataFrame
        columns = [
            "time",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "close_time",
            "quote_volume",
            "trades",
            "taker_buy_base",
            "taker_buy_quote",
            "ignore",
        ]
        df = pd.DataFrame(klines, columns=columns, dtype=float)

        # Конвертация времени в читаемый формат
        df["time"] = pd.to_datetime(df["time"], unit="ms")

        return df[["time", "open", "high", "low", "close", "volume", "trades"]]

    except Exception as e:
        print(f"Ошибка: {e}")
        return None


# Получить данные за последние 24 часа (1-минутные, 1- секундные свечи)
# df = get_klines(symbol="ETHUSDT", interval="1s")
df = get_klines(symbol="ETHUSDT", interval="1m")
# Вывод первых 5 строк
print(df.head())

                 time     open     high      low    close     volume  trades
0 2025-03-17 14:38:00  1902.73  1906.57  1902.64  1904.84   312.7654  2639.0
1 2025-03-17 14:39:00  1904.84  1907.83  1904.84  1905.86   489.3576  2253.0
2 2025-03-17 14:40:00  1905.86  1906.87  1904.34  1904.87  1222.1563  2289.0
3 2025-03-17 14:41:00  1904.88  1904.96  1902.65  1903.61  1085.9549  2013.0
4 2025-03-17 14:42:00  1903.61  1904.37  1903.36  1903.95   177.1586  1032.0


In [11]:
N = len(df)
ratio = 0.8
df_test = df[int(N * ratio) :]
df = df[: int(N * ratio)]


В этом подходе мы настраиваем параметр гамма с помощью RL для максимизации PnL

### Среда
- **Действие**: выбор $\gamma$ из квантовонного набора
- **Наблюдение**: inventory, относительное изменение цены (return), PnL

In [ ]:
class MarketMakingEnv(gym.Env):
    """
    Окружение маркет-мейкинга по Avellaneda-Stoikov без учета задержки.
    Состояние: [inventory, относительное изменение цены, PnL]
    Действие: выбор индекса для γ из заданного списка (например, [0.01, 0.1, 0.5, 1.0])

    #TODO
    - kappa - параметра интенсивности, определяется из книги заказов
    """

    def __init__(self, df, gamma_values, T=60.0, kappa=1e-3):
        super(MarketMakingEnv, self).__init__()
        self.prices = df.close.values
        self.prices_high = df.high.values
        self.prices_low = df.low.values
        self.N = len(df)
        self.gamma_values = gamma_values
        self.T = T  # горизонт торгов (сек)
        self.kappa = kappa  # параметр ликвидности
        # Определяем пространства:
        # Действие – дискретный выбор из len(gamma_values)
        self.action_space = spaces.Discrete(len(gamma_values))
        # Состояние: [inventory, relative price change, PnL]
        high = np.array([100.0, 1.0, 1e9], dtype=np.float32)
        low = np.array([-100.0, -1.0, -1e9], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)
        self.initial_cash = 10000.0
        self.fee_rate = 0.0  # можно задать комиссию

    def reset(self):
        self.t = 0
        self.inventory = 0.0
        self.cash = self.initial_cash
        self.prev_price = self.prices[self.t]
        # Начальное состояние: инвентарь 0, изменения цены = 0, PnL = 0
        obs = np.array([self.inventory, 0.0, 0.0], dtype=np.float32)
        return obs

    def step(self, action):
        gamma = self.gamma_values[action]
        mid_price = self.prices[self.t]
        # Оценка волатильности на окне последних 60 минут
        sigma = 0.0
        if self.t > 1:
            start = max(0, self.t - 60)
            window_prices = self.prices[start : self.t + 1]
            if len(window_prices) > 1:
                returns = np.diff(window_prices) / np.array(window_prices[:-1])
                sigma = np.std(returns)
        # Вычисляем цену резервирования и ширину спреда
        reservation_price = mid_price - self.inventory * gamma * (sigma**2) * self.T
        delta = gamma * (sigma**2) * self.T + (1.0 / gamma) * math.log(
            1 + gamma / self.kappa
        )

        bid_price = reservation_price - delta / 2
        ask_price = reservation_price + delta / 2
        done = False
        reward = 0.0
        if self.t < self.N - 1:
            next_price = self.prices[self.t + 1]
            # Если цена выше ask – исполняется заявка на продажу (продажа 1 единицы)
            if self.prices_high[self.t + 1] >= ask_price:
                self.inventory -= 1.0
                self.cash += ask_price * (1.0 - self.fee_rate)
            # Если цена ниже bid – исполняется заявка на покупку (покупка 1 единицы)
            if self.prices_low[self.t + 1] <= bid_price:
                self.inventory += 1.0
                self.cash -= bid_price * (1.0 + self.fee_rate)
            current_value = self.cash + self.inventory * next_price
            prev_value = self.cash + self.inventory * mid_price
            pnl_change = current_value - prev_value
            # Небольшой штраф за большую позицию
            reward = pnl_change - 0.001 * abs(self.inventory)
            self.t += 1
        else:
            done = True
            current_value = self.cash + self.inventory * mid_price
            reward = current_value - self.initial_cash
        if not done:
            new_mid = self.prices[self.t]
            price_change = (new_mid - mid_price) / mid_price
            pnl = self.cash + self.inventory * new_mid - self.initial_cash
            obs = np.array([self.inventory, price_change, pnl], dtype=np.float32)
        else:
            obs = None
        return obs, reward, done, {}


### Агенты
Используем DQN и A2C

In [ ]:
# ==========================
# DQN-агент

# Определяем простую Q-сеть
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


Transition = namedtuple(
    "Transition", ["state", "action", "reward", "next_state", "done"]
)


class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, *args):
        self.buffer.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)


class DQNAgent:
    def __init__(
        self,
        state_dim,
        action_dim,
        lr=1e-3,
        gamma=0.99,
        buffer_capacity=10000,
        batch_size=64,
        target_update=1000,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.q_network = QNetwork(state_dim, action_dim).to(self.device)
        self.target_network = QNetwork(state_dim, action_dim).to(self.device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.gamma = gamma
        self.buffer = ReplayBuffer(buffer_capacity)
        self.batch_size = batch_size
        self.target_update = target_update
        self.steps_done = 0
        self.action_dim = action_dim

    def select_action(self, state, epsilon):
        if random.random() < epsilon:
            return random.randrange(self.action_dim)
        else:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.q_network(state_tensor)
            return q_values.argmax().item()

    def push_transition(self, state, action, reward, next_state, done):
        self.buffer.push(state, action, reward, next_state, done)

    def update(self):
        if len(self.buffer) < self.batch_size:
            return
        transitions = self.buffer.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        state_batch = torch.FloatTensor(batch.state).to(self.device)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch.reward).unsqueeze(1).to(self.device)
        non_final_mask = torch.tensor(
            [s is not None for s in batch.next_state],
            dtype=torch.bool,
            device=self.device,
        )
        non_final_next_states = torch.FloatTensor(
            [s for s in batch.next_state if s is not None]
        ).to(self.device)
        done_batch = torch.FloatTensor(batch.done).unsqueeze(1).to(self.device)
        # Q(s,a)
        q_values = self.q_network(state_batch).gather(1, action_batch)
        # Q_target
        next_q_values = torch.zeros(self.batch_size, 1).to(self.device)
        if non_final_next_states.size(0) > 0:
            next_q_values[non_final_mask] = self.target_network(
                non_final_next_states
            ).max(1, keepdim=True)[0]
        expected_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values
        loss = nn.MSELoss()(q_values, expected_q_values.detach())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.steps_done += 1
        if self.steps_done % self.target_update == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())


# ==========================
# A2C-агент


class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.actor = nn.Linear(64, action_dim)
        self.critic = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        logits = self.actor(x)
        value = self.critic(x)
        return logits, value


class A2CAgent:
    def __init__(
        self,
        state_dim,
        action_dim,
        lr=1e-3,
        gamma=0.99,
        value_coef=0.5,
        entropy_coef=0.01,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = ActorCritic(state_dim, action_dim).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef

    def select_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        logits, value = self.model(state_tensor)
        probs = torch.softmax(logits, dim=1)
        action = torch.multinomial(probs, num_samples=1)
        log_prob = torch.log(probs.gather(1, action))
        return action.item(), log_prob.squeeze(), value.squeeze(), logits.squeeze()

    def update(self, trajectories):
        # trajectories: список кортежей (state, action, log_prob, value, reward, done, logits)

        log_probs = torch.stack([t[2] for t in trajectories]).to(self.device)
        values = torch.stack([t[3] for t in trajectories]).to(self.device)
        rewards = [t[4] for t in trajectories]
        dones = [t[5] for t in trajectories]
        logits_list = torch.stack([t[6] for t in trajectories]).to(self.device)

        # Вычисляем кумулятивную сумму вознаграждений (returns) обратным проходом
        R = 0
        returns = []
        for reward, done in zip(reversed(rewards), reversed(dones)):
            R = reward + self.gamma * R * (1 - done)
            returns.insert(0, R)
        returns = torch.FloatTensor(returns).to(self.device)
        advantages = returns - values
        actor_loss = -(log_probs * advantages.detach()).mean()
        critic_loss = advantages.pow(2).mean()
        probs = torch.softmax(logits_list, dim=1)
        entropy = -(probs * torch.log(probs + 1e-10)).sum(dim=1).mean()
        loss = actor_loss + self.value_coef * critic_loss - self.entropy_coef * entropy
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


### Обучение

In [23]:
# Задаём параметры окружения и агентов
gamma_values = [0.005, 0.01, 0.05, 0.1, 0.5, 1.0]
env = MarketMakingEnv(df=df, gamma_values=gamma_values, T=60.0, kappa=1e-3)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Параметры для обучения DQN
dqn_agent = DQNAgent(
    state_dim,
    action_dim,
    lr=1e-3,
    gamma=0.99,
    buffer_capacity=10000,
    batch_size=64,
    target_update=1000,
)


num_episodes = 20
num_episodes_a2c = 20
epsilon_start = 1.0
epsilon_final = 0.1
epsilon_decay = 300

print("Обучение DQN-агента...")
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        # ε-жадная политика
        epsilon = epsilon_final + (epsilon_start - epsilon_final) * math.exp(
            -env.t / epsilon_decay
        )
        action = dqn_agent.select_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        dqn_agent.push_transition(state, action, reward, next_state, done)
        dqn_agent.update()
        state = next_state if next_state is not None else state
        total_reward += reward
    print(
        f"Эпизод {episode+1}/{num_episodes}: суммарное вознаграждение = {total_reward:.2f}"
    )

# Обучение A2C-агента

a2c_agent = A2CAgent(
    state_dim, action_dim, lr=1e-3, gamma=0.99, value_coef=0.5, entropy_coef=0.01
)


print("\nОбучение A2C-агента...")

rollout_length = 10  # собираем траектории по 10 шагов
for episode in range(num_episodes_a2c):
    state = env.reset()
    trajectories = []
    total_reward = 0
    done = False
    while not done:
        # Собираем траекторию rollout_length
        for _ in range(rollout_length):
            action, log_prob, value, logits = a2c_agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectories.append(
                (state, action, log_prob, value, reward, float(done), logits)
            )
            total_reward += reward
            if done:
                break
            state = next_state
        a2c_agent.update(trajectories)
        trajectories = []
    print(
        f"Эпизод {episode+1}/{num_episodes_a2c}: суммарное вознаграждение = {total_reward:.2f}"
    )


Обучение DQN-агента...
Эпизод 1/20: суммарное вознаграждение = 338.21
Эпизод 2/20: суммарное вознаграждение = 173.55
Эпизод 3/20: суммарное вознаграждение = 57.82
Эпизод 4/20: суммарное вознаграждение = 873.25
Эпизод 5/20: суммарное вознаграждение = 1368.27
Эпизод 6/20: суммарное вознаграждение = 22.46
Эпизод 7/20: суммарное вознаграждение = -779.97
Эпизод 8/20: суммарное вознаграждение = -169.03
Эпизод 9/20: суммарное вознаграждение = -102.55
Эпизод 10/20: суммарное вознаграждение = 866.24
Эпизод 11/20: суммарное вознаграждение = -363.19
Эпизод 12/20: суммарное вознаграждение = 495.31
Эпизод 13/20: суммарное вознаграждение = 470.04
Эпизод 14/20: суммарное вознаграждение = 837.97
Эпизод 15/20: суммарное вознаграждение = 294.30
Эпизод 16/20: суммарное вознаграждение = -575.31
Эпизод 17/20: суммарное вознаграждение = -243.46
Эпизод 18/20: суммарное вознаграждение = -409.54
Эпизод 19/20: суммарное вознаграждение = -174.26
Эпизод 20/20: суммарное вознаграждение = 425.28

Обучение A2C-агент

### Тест

In [24]:
def run_agent(env, agent, agent_type="dqn"):
    state = env.reset()
    pnl_history = []
    done = False
    while not done:
        if agent_type == "dqn":
            action = agent.select_action(state, epsilon=0.0)
        elif agent_type == "a2c":
            action, _, _, _ = agent.select_action(state)
        else:
            action = 0  # базовая стратегия
        next_state, _, done, _ = env.step(action)
        # Третий элемент состояния – PnL
        if state is not None:
            pnl = state[2]
            pnl_history.append(pnl)
        state = next_state if next_state is not None else state
    return pnl_history


# Прогон теста для обоих агентов
env_test = MarketMakingEnv(df=df_test, gamma_values=gamma_values, T=60.0, kappa=1e-3)
dqn_pnl = run_agent(env_test, dqn_agent, agent_type="dqn")
env_test = MarketMakingEnv(df=df_test, gamma_values=gamma_values, T=60.0, kappa=1e-3)
a2c_pnl = run_agent(env_test, a2c_agent, agent_type="a2c")


In [25]:
def compute_max_drawdown(pnl_history):
    pnl_array = np.array(pnl_history)
    running_max = np.maximum.accumulate(pnl_array)
    drawdowns = pnl_array - running_max
    max_drawdown = drawdowns.min()
    return max_drawdown


def compute_sharpe_ratio(pnl_history):
    pnl_array = np.array(pnl_history)
    returns = np.diff(pnl_array)
    if returns.std() == 0:
        return 0.0
    sharpe = returns.mean() / returns.std() * np.sqrt(len(returns))
    return sharpe


dqn_max_dd = compute_max_drawdown(dqn_pnl)
dqn_sharpe = compute_sharpe_ratio(dqn_pnl)
a2c_max_dd = compute_max_drawdown(a2c_pnl)
a2c_sharpe = compute_sharpe_ratio(a2c_pnl)

print("\nРезультаты тестирования:")
print(
    f"DQN итоговый PnL: {dqn_pnl[-1]:.2f}, Max Drawdown: {dqn_max_dd:.2f}, Sharpe Ratio: {dqn_sharpe:.2f}"
)
print(
    f"A2C итоговый PnL: {a2c_pnl[-1]:.2f}, Max Drawdown: {a2c_max_dd:.2f}, Sharpe Ratio: {a2c_sharpe:.2f}"
)


Результаты тестирования:
DQN итоговый PnL: 0.00, Max Drawdown: 0.00, Sharpe Ratio: 0.00
A2C итоговый PnL: -8.07, Max Drawdown: -52.28, Sharpe Ratio: -0.18


Выводы:  
С задержкой и без задержки алгоритмы показали примерно одинаковые, близкие к случайным  результаты  